# Baseline (01)

# Imports

In [ ]:
!pip install gdown

In [ ]:
import pandas as pd
import numpy as np
import os
import gdown

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa


#from wcs.google import google_drive_share
import pandas as pd
import urllib.request
from urllib.parse import urlparse

#from google.colab import drive


import warnings
warnings.simplefilter(action='ignore')
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Load Data

In [ ]:
BASE_DIR ="./"
#IMAGES_DIR = "/kaggle/input/movie-poster-genre-2021/images/"
IMAGES_DIR = "/kaggle/input/moviepostergenre20212/images/images/"

In [ ]:
#!ls $IMAGES_DIR

# Preproc

In [ ]:
#https://drive.google.com/file/d/1MlXZKtRUP7pOehDR9j5MMvBNXF4UNNUx/view?usp=sharing
url = 'https://drive.google.com/uc?id=1MlXZKtRUP7pOehDR9j5MMvBNXF4UNNUx'
holdout_fname = "df_holdout.csv"
if not os.path.exists(holdout_fname):
    gdown.download(url, holdout_fname, quiet=False)
    df_holdout = pd.read_csv(holdout_fname,sep=";")
    df_holdout.head()

#Bitte diese 1000 Movie IDs vom Training und vom gesamten Model Engineering ausschließen und 
#nur für die Einträge im Leaderboard verwenden.

In [ ]:
def retrieve_filename(url):
    a = urlparse(url)
    picfilename = os.path.basename(a.path)
    return picfilename


In [ ]:
parquet_fname = "./df.parquet.gzip"
os.path.exists(parquet_fname)
#!rm $parquet_fname

In [ ]:
url = 'https://drive.google.com/uc?id=1ynvEZMYRonM2SuJ9mQ4kCkznMW6dsqcA'
gdown.download(url, parquet_fname, quiet=False)
df = pd.read_parquet(parquet_fname)
df['genre_id'] = df['genre_id'].apply(lambda x: list(x))

df.head()

In [ ]:
%%time
#https://drive.google.com/file/d/178mXeXUC1400lj-LrWyhD3yfW5_KsVYw/view?usp=sharing
if not os.path.exists(parquet_fname):
    url = 'https://drive.google.com/uc?id=178mXeXUC1400lj-LrWyhD3yfW5_KsVYw'
    gdown.download(url, parquet_fname, quiet=False)
    df = pd.read_parquet(parquet_fname)
    
    df["filename"] = df.loc[~df["poster_url"].isnull(),"poster_url"].apply(retrieve_filename)
    
    # use only mot null rows
    df = df.dropna()
    # set data path to basename of the file
    #df['data_path'] = df['poster_path'].apply(lambda x: x.split('/')[-1])
    # remove rows with empty genre_id list and set correct list type
    df['genre_id'] = df['genre_id'].apply(lambda x: np.nan if len(eval(x)) == 0 else x)
    #df = df.dropna()
    df['genre_id'] = df['genre_id'].apply(lambda x: eval(x))

    print(f'len of df: {len(df)}')
    print("Checking existence of images")
    df["file_exists"] = df["filename"].apply(lambda x: os.path.exists(IMAGES_DIR + x))
    print("Flagging hold out images")
    df["is_holdout"] = False
    df.loc[df["id"].isin(df_holdout["id"]),"is_holdout"] = True

    df.to_parquet(parquet_fname,compression='gzip')
else:
    df = pd.read_parquet("./df.parquet.gzip")

In [ ]:
df['genre_id'] = df['genre_id'].apply(lambda x: list(x))

type(df["genre_id"].iloc[2])

In [ ]:
df.is_holdout.value_counts()

In [ ]:
df.file_exists.value_counts()
#print(df.shape)

In [ ]:
#keep only rows where file exists in data set
df = df.loc[df["file_exists"]]

In [ ]:
df.loc[~df.is_holdout].shape

In [ ]:
df.loc[df.is_holdout].shape

In [ ]:
df.loc[df.is_holdout].head()

 Create ImageGenerators

In [ ]:
IMAGES_DIR

In [ ]:
%%time
datagen = ImageDataGenerator(rescale=1 / 255.)#, validation_split=0.1)
BATCH_SIZE = 64
train_generator = datagen.flow_from_dataframe(
    dataframe=df.loc[~df.is_holdout],
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",    
    target_size=(299, 299),
    subset='training',
    validate_filenames=False
)


In [ ]:
%%time
#datagen = ImageDataGenerator(rescale=1 / 255.)#, validation_split=0.1)
valid_generator = datagen.flow_from_dataframe(
    dataframe=df.loc[df.is_holdout],
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(299, 299),
    subset='training',
    validate_filenames=False
)

%%time
datagen = ImageDataGenerator(rescale=1 / 255., validation_split=0.1)
BATCH_SIZE = 64
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",    
    target_size=(299, 299),
    subset='training',
    validate_filenames=True
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col="genre_id",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(299, 299),
    subset='validation',
    validate_filenames=True
)

In [ ]:
# show class indicies
print(train_generator.class_indices)
print('length:', len(train_generator.class_indices))


In [ ]:
list(train_generator.class_indices.keys())

In [ ]:
#https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras
from sklearn.utils import class_weight
#In order to calculate the class weight do the following

class_weights = class_weight.compute_class_weight('balanced',
                                                  np.array(list(train_generator.class_indices.keys()),dtype="int"), 
                                                  np.array(df.genre_id.explode(),dtype="int"))

class_weights_genre_id = dict(zip(list(train_generator.class_indices), class_weights))
display(class_weights_genre_id)
class_weights = dict(zip(list(range(len(class_weights))), class_weights))
class_weights

In [ ]:
map_gender={"28":"Action",
"12":"Adventure",
"16":"Animation",
"35":"Comedy",
"80":"Crime",
"99":"Documentary",
"18":"Drama",
"10751":"Family",
"14":"Fantasy",
"36":	"History",
"27":"Horror",
"10402"	:"Music",
"9648":"Mystery",
"10749":"Romance",
"878"	:"Science Fiction",
"10770":"TV Movie",
"53":"Thriller",
"10752":"War",
"37":"Western"}

series_genre_id_counts = df.genre_id.explode().value_counts()
series_genre_id_counts
df_genre = pd.DataFrame(series_genre_id_counts)
df_genre["id"] = df_genre.index
df_genre.rename(columns={"genre_id" : "count"},inplace=True)
df_genre["name"] = df_genre["id"].apply(lambda x : map_gender[str(x)])
df_genre["weight"] = df_genre["id"].apply(lambda x : class_weights_genre_id[x])
df_genre.sort_values(by="count")

# Simple Model

In [ ]:
model = keras.Sequential(
    [
      layers.Conv2D(32, (3, 3), padding='same', input_shape=(299, 299, 3)),
      layers.Activation('relu'),
      layers.Conv2D(32, (3, 3)),
      layers.Activation('relu'),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Dropout(0.25),
      layers.Conv2D(64, (3, 3), padding='same'),
      layers.Activation('relu'),
      layers.Conv2D(64, (3, 3)),
      layers.Activation('relu'),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Dropout(0.25),
      layers.Flatten(),
      layers.Dense(512),
      layers.Activation('relu'),
      layers.Dropout(0.5),
      layers.Dense(len(train_generator.class_indices), activation='sigmoid')
    ]
)


#model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])

model = keras.Sequential(
    [ 
        layers.Conv2D(32, (3, 3), padding='same', input_shape=(299, 299, 3)),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),
        layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(len(train_generator.class_indices), activation='sigmoid')
        #layers.Dense(len(train_generator.class_indices), activation='softmax')

    ])

In [ ]:
#model = Model(inputs=model.inputs, outputs=classifications)
model.summary()

In [ ]:
f1_micro = tfa.metrics.F1Score(num_classes=19, average='micro', name='f1_micro'), 
f1_macro = tfa.metrics.F1Score(num_classes=19, average='macro', name='f1_macro')
f1_weighted = tfa.metrics.F1Score(num_classes=19,  average='weighted', name='f1_score_weighted')

In [ ]:
import os.path

fname_model = BASE_DIR + "CNN_Simple_1"

if  os.path.exists(fname_model) :
    print("Load model")
    model = keras.models.load_model(fname_model, custom_objects={'_tf_keras_metric':f1_micro})


In [ ]:
from keras import metrics
#https://neptune.ai/blog/keras-metrics
model.compile(optimizer='adam', loss="binary_crossentropy", 
              metrics=["categorical_accuracy",
                       tf.keras.metrics.AUC(multi_label = True),#,label_weights=class_weights),
                        f1_micro,
                        f1_macro,
                        f1_weighted,
                      ])

# Train

In [ ]:
!mkdir model_checkpoints

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = "model_checkpoints/saved-model-{epoch:02d}-{val_f1_score_weighted:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_f1_score_weighted', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,# 5,
    #batch_size=BATCH_SIZE,
    #steps_per_epoch=10,#1430,
    class_weight = class_weights,
    callbacks=callbacks_list
)

In [ ]:
!mkdir final_model
!mkdir final_model_weights

In [ ]:
print("Saving final model")
model.save("final_model/" + fname_model)
print("Saving final model weights")
model.save_weights("final_model_weights/" + fname_model + ".ckpt")
#load_status = sequential_model.load_weights("ckpt")